
# **Modeling Flu Spread in Bucharest using Monte Carlo Simulation** #


##### The aim of this project is to model the spread of the flu in Bucharest using the Monte Carlo method to understand the dynamics of a virus in a crowded city. #####

In [670]:
import numpy as np
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

##### In this simulation model for Bucharest, we used a sample population of 10,000 individuals, with an estimated probability of infection between 3-5% per contact and an average recovery time of 7-10 days. The simulation is carried out over a 90-day period to cover a typical influenza season, allowing to observe the evolution of the spread of infection in a large city. ##### 

In [671]:
# Parameters for the simulation
nr_populatie = 10000
prob_infectare = np.random.uniform(0.03, 0.05, nr_populatie)  # Probability of infection per individual
nr_zile_refacere = np.zeros(nr_populatie, dtype=int)          # Set initial recovery days to 0
simulare_zile = 150                                           # Number of simulation days (1 nov - 31 mar)
nr_sectoare = 6

In [672]:
# Simulation of the spread of the virus in the population every day
nr_infectati_pe_zi = []

for zi in range(simulare_zile):
    posibil_infectati = nr_zile_refacere == 0  # Not currently recovering
    sansa_infectare = np.random.rand(nr_populatie) 
    infectati = posibil_infectati & (sansa_infectare < prob_infectare)  # If random number is less than infection probability
    nr_infectati_pe_zi.append(np.sum(infectati))

    # Set recovery days for newly infected individuals
    nr_zile_refacere[infectati] = np.random.randint(7, 11, size=np.sum(infectati))

    # Decrease recovery days for all those recovering
    nr_zile_refacere[nr_zile_refacere > 0] -= 1

# Display results
print("Nr infectati pe zi: ", nr_infectati_pe_zi)


Nr infectati pe zi:  [394, 353, 407, 376, 329, 307, 313, 345, 307, 289, 324, 295, 304, 301, 322, 286, 311, 303, 287, 319, 329, 320, 281, 289, 294, 320, 312, 264, 319, 286, 302, 309, 317, 318, 290, 315, 331, 309, 302, 310, 290, 318, 300, 309, 301, 314, 329, 284, 295, 291, 323, 288, 322, 320, 282, 315, 284, 301, 343, 309, 309, 313, 324, 309, 279, 291, 297, 309, 303, 298, 317, 284, 322, 306, 326, 319, 336, 306, 342, 297, 289, 275, 340, 323, 313, 318, 285, 313, 329, 297, 300, 294, 278, 320, 302, 315, 334, 304, 321, 299, 317, 285, 315, 293, 362, 300, 324, 274, 302, 324, 280, 283, 297, 305, 276, 298, 318, 333, 313, 312, 291, 308, 335, 313, 282, 335, 311, 272, 305, 309, 324, 298, 288, 275, 311, 309, 298, 317, 285, 350, 299, 280, 317, 292, 295, 321, 319, 307, 344, 305]


In [673]:
# plotarea evolutiei numarului de infectati in fiecare zi
zile_pe_luna = [30, 31, 31, 28, 31]
luni = ['Nov', 'Dec', 'Jan', 'Feb', 'Mar'] 
pozitie_luna = [i * zile_pe_luna[i] for i in range(len(zile_pe_luna))]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(simulare_zile), 
    y=nr_infectati_pe_zi, 
    mode='lines', 
    name='Numar infectati in perioda noiembrie - martie', 
    line=dict(color='MediumPurple', width=2)))

fig.update_layout(title=dict(text='Evolutia gripei in Bucuresti (noiembrie - martie)', x=0.5),
                   xaxis=dict(title='Zile', tickvals=pozitie_luna, ticktext=luni),
                   yaxis_title='Nr persoane infectate',
                   paper_bgcolor='rgba(0,0,0,0)',
                   font=dict(family='system-ui', size=14, color='white'),
                   plot_bgcolor='Gainsboro', )
fig.show()

In [674]:
# Simulation parameters per sector
populatie_simulata_sectoare = np.array([1192, 1745, 2244, 1546, 1422, 1849])  # Simulated population for each sector
prob_infectare_sectoare = np.random.uniform(0.03, 0.05, nr_sectoare)          # Infection probability per sector

# Initialize recovery arrays for each sector
nr_zile_refacere_sectoare = [np.zeros(pop, dtype=int) for pop in populatie_simulata_sectoare]

# List to hold the number of infected individuals per day for each sector
nr_infectati_pe_zi_sectoare = [[] for _ in range(nr_sectoare)]

for zi in range(simulare_zile):
    for s in range(nr_sectoare):
        # Determine those who are susceptible to infection (not recovering)
        posibil_infectati = nr_zile_refacere_sectoare[s] == 0
        sansa_infectare = np.random.rand(populatie_simulata_sectoare[s])

        # Determine new infections
        infectati = posibil_infectati & (sansa_infectare < prob_infectare_sectoare[s])
        nr_infectati_pe_zi_sectoare[s].append(np.sum(infectati))

        # Set recovery days for newly infected individuals
        nr_zile_refacere_sectoare[s][infectati] = np.random.randint(7, 11, size=np.sum(infectati))

        # Decrement recovery days for those who are still in recovery
        nr_zile_refacere_sectoare[s][nr_zile_refacere_sectoare[s] > 0] -= 1

# Display results
for s in range(nr_sectoare):
    print(f"Nr infectati pe zi in sectorul {s+1}: {nr_infectati_pe_zi_sectoare[s]}")


Nr infectati pe zi in sectorul 1: [44, 41, 37, 44, 26, 30, 32, 41, 27, 30, 33, 41, 32, 37, 29, 36, 44, 30, 29, 35, 27, 32, 35, 37, 44, 35, 40, 41, 40, 34, 34, 34, 41, 39, 34, 36, 27, 28, 33, 35, 24, 32, 30, 35, 37, 41, 35, 29, 30, 24, 32, 41, 28, 42, 27, 41, 37, 39, 28, 39, 33, 28, 43, 32, 30, 32, 28, 34, 20, 28, 39, 32, 27, 30, 42, 34, 37, 35, 26, 27, 35, 31, 29, 33, 33, 27, 46, 38, 36, 31, 27, 33, 31, 34, 39, 46, 33, 31, 28, 31, 44, 31, 30, 22, 34, 37, 38, 30, 40, 39, 33, 35, 32, 30, 42, 28, 45, 27, 36, 22, 47, 40, 38, 36, 27, 33, 41, 26, 32, 30, 28, 30, 32, 30, 33, 42, 30, 35, 34, 42, 28, 35, 43, 33, 33, 36, 34, 27, 38, 29]
Nr infectati pe zi in sectorul 2: [73, 67, 71, 67, 68, 62, 68, 56, 56, 68, 57, 66, 56, 67, 45, 54, 57, 52, 48, 58, 70, 75, 64, 58, 67, 56, 64, 40, 56, 59, 50, 68, 57, 58, 54, 58, 51, 59, 72, 52, 50, 48, 50, 69, 63, 55, 58, 64, 53, 54, 60, 64, 56, 66, 57, 59, 62, 49, 55, 61, 61, 58, 65, 67, 51, 72, 43, 59, 72, 62, 59, 58, 66, 52, 54, 71, 70, 46, 67, 53, 72, 73, 63

In [675]:
infectii_pe_sector_toate_zilele = [sum(nr_infectati_pe_zi_sectoare[s]) for s in range(nr_sectoare)]

# Define sector labels that match the GeoJSON
idx_sector = ['Sector 1', 'Sector 2', 'Sector 3', 'Sector 4', 'Sector 5', 'Sector 6']

# Load the GeoJSON file
geojson_fisier = '/Users/alexandrazamfirescu/TS/BucurestiSectoare.geo.json'
with open(geojson_fisier) as f:
    geojson = json.load(f)

# Create a DataFrame with the sector labels and infection counts
data = pd.DataFrame({'Sector': idx_sector, 'Infectii': infectii_pe_sector_toate_zilele})
print(data)

# # Plot the choropleth map
fig = px.choropleth(
    data_frame=data,
    geojson=geojson,
    locations='Sector',
    featureidkey='properties.tags.name',  # Ensure this matches your GeoJSON
    color='Infectii',
    color_continuous_scale='Turbo',
    labels={'Infectii': 'Nr infectati'},
    projection='mercator'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title=dict(text='Nivelul de raspandire al gripei in fiecare sector', x=0.5),
    margin={"r":0, "t":40, "l":0, "b":0}
)
fig.show()

     Sector  Infectii
0  Sector 1      5081
1  Sector 2      8934
2  Sector 3     11094
3  Sector 4      8076
4  Sector 5      5917
5  Sector 6      9295
